Why semi-supervised learning?

In [2]:
import os
os.chdir("../../../")

https://stats.stackexchange.com/questions/321841/what-are-variational-autoencoders-and-to-what-learning-tasks-are-they-used

Ideal parameters:
$$ \min_{\boldsymbol{\lambda}}\mathcal{D}[p(\mathbf{z}\vert \mathbf{x})\vert\vert q(\mathbf{z}\vert \mathbf{x},\boldsymbol{\lambda})] $$

It should also minimize the reconstruction loss (and optional regularization terms, mainly L1 or L2)

VAE loss function:
$$ELBO(\boldsymbol{\lambda})= E_{q(\boldsymbol{z}\vert \mathbf{x},\boldsymbol{\lambda})}[\log p(\mathbf{x}\vert\boldsymbol{z})]-\mathcal{D}[(q(\boldsymbol{z}\vert \mathbf{x},\boldsymbol{\lambda})\vert\vert p(\boldsymbol{z})]$$

$$ q(\mathbf{z}\vert \mathbf{x},\boldsymbol{\lambda}) = \mathcal{N}(\mathbf{z}\vert\boldsymbol{\mu}(\mathbf{x}), \boldsymbol{\sigma}^2(\mathbf{x})I) $$

conditional distribution:
$$ p_{\boldsymbol{\phi}}(\mathbf{x}\vert\mathbf{z}) = \mathcal{N}(\mathbf{x}|\boldsymbol{\mu}(\mathbf{z}; \boldsymbol{\phi}), \boldsymbol{\sigma}(\mathbf{z}; \boldsymbol{\phi})^2I)$$


$$ ELBO(\boldsymbol{\theta},\boldsymbol{\phi})= \sum_i E_{q_{\boldsymbol{\theta}}(\boldsymbol{z}\vert \mathbf{x}_i,\boldsymbol{\lambda})}[\log p_{\boldsymbol{\phi}}(\mathbf{x}_i\vert\boldsymbol{z})]-\mathcal{D}[(q_{\boldsymbol{\theta}}(\boldsymbol{z}\vert \mathbf{x}_i,\boldsymbol{\lambda})\vert\vert p(\boldsymbol{z})] $$

In [1]:
# Import packages
#from models.dimension_reduction.ordination import ordination2d
#from sklearn.decomposition import PCA
from IPython.display import Image
import pandas as pd
import numpy as np
# Import models
from models.generative.autoencoders.vae.vae import VariationalAutoencoder
from models.generative.autoencoders.vae.sylvester_vae import SylvesterVAE
from models.utils.utils import dict_of_int_highest_elements, plot_evaluation



ImportError: cannot import name 'log_gaussian'

In [3]:
# files_destinations
home_path = "/home/simon/"
destination_folder = "annleukemia"
data_folder = "data"
results_folder = "results"
meta_destination_folder = "pandas_meta_df"

plots_folder_path = "/".join([home_path, destination_folder, results_folder, "plots/"])

#dataset_name = "gse33000_and_GSE24335_GSE44768_GSE44771_GSE44770"
dataset_name = "mnist"
activation = "relu"
#nrep = 3
betas=(0.9, 0.999)
vae_flavour = "o-sylvester"
early_stopping = 200
labels_per_class = 1000
n_epochs = 100
warmup = 0
gt_input = 0

# if ladder is yes builds a ladder vae. Do not combine with auxiliary (yet; might be possible and relatively 
# not too hard to implement, but might be overkill. Might be interesting too)
translate = "n" 

# Convolution neural network (convolutional VAE and convolutional classifier)
use_conv = False #Not applicable if not sequence (images, videos, sentences, DNA...)

# Ladder VAE (L-VAE)
ladder = False
# Load pre-computed vae (unsupervised learning)
load_vae = False


In [4]:
lr = 1e-4
l1 = 0.
l2 = 0.
batch_size = 128
mc = 1 # seems to be a problem when mc > 1 for display only, results seem good
iw = 1 # seems to be a problem when iw > 1 for display only, results seem good

# Neurons layers
h_dims = [128, 64]
z_dims = [50]

# number of flows
number_of_flows = 8
num_elements = 3

is_example = True


In [5]:
if ladder:
    vae = LadderVariationalAutoencoder(vae_flavour, z_dims=z_dims, h_dims=h_dims, n_flows=number_of_flows,
                                       num_elements=num_elements)
    z_dim = z_dims[-1]
elif vae_flavour in ["o-sylvester", "h-sylvester", "t-sylvester"]:
    print("vae_flavour", vae_flavour)
    vae = SylvesterVAE(vae_flavour, z_dims=z_dims, h_dims=h_dims, n_flows=number_of_flows,
                       num_elements=num_elements, auxiliary=False, a_dim=0)
else:
    print("vae_flavour", vae_flavour)
    vae = VariationalAutoencoder(vae_flavour, z_dim=z_dims, h_dims=h_dims, n_flows=number_of_flows, auxiliary=False, a_dim=0)
    
vae.load_example_dataset(dataset="mnist", batch_size=batch_size, labels_per_class=0, extra_class=True, 
                         unlabelled_train_ds=None, normalize=True, mu=0.1307, var=0.3081, unlabelled_samples=False)

train = np.vstack([x[0].data.numpy() for x in vae.x_train])
#unlabelled_train = np.vstack([x[0].data.numpy() for x in dgm.unlabelled_x_train])

targets = np.vstack([x[1].data.numpy() for x in vae.x_train])
labels = [x.tolist().index(1) for x in targets]
vae.set_configs(home_path=home_path, results_folder=results_folder, data_folder=data_folder,
               destination_folder=destination_folder, dataset_name=dataset_name, lr=lr,
               meta_destination_folder="meta_pandas_dataframes", csv_filename="csv_loggers")

vae.define_configurations(vae_flavour, early_stopping=1000, warmup=warmup, ladder=ladder, z_dim=z_dims[-1], auxiliary=False, ssl=False)

vae.set_data(is_example=is_example, labels_per_class=0)
if ladder:
    print("Setting ladder layers")
    vae.set_lvae_layers()
else:
    vae.set_vae_layers()

if load_vae:
    vae.load_model()

vae.run(epochs=n_epochs, clip_grad=1e-4, gen_rate=1)

vae_flavour o-sylvester
a_dim (making sure it stays ok for ssl_vae) 0
self.a_dim 0


/home/simon/anaconda3/lib/python3.6/site-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


No unlabelled data
self.train_loader 468
self.train_loader 128
self.train_loader <torch.utils.data.sampler.RandomSampler object at 0x7f0230dca518>
create model
beta: 1.0
Labelled 468


/home/simon/anaconda3/lib/python3.6/site-packages/torch/tensor.py:287: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


EVALUATION!
Generating images
GENERATING RANDOM IMAGES autoencoder!
Images location: /home/simon//annleukemia/results/images/examples/generative/not_ladder/o-sylvester/
GENERATING RECONSTRUCTION IMAGES autoencoder!


RuntimeError: shape '[-1, 1, 28, 28]' is invalid for input of size 1

In [ ]:
#print("PCA saved at: ", plots_folder_path)
#meta_df = pd.DataFrame(train.transpose(), columns=labels)
#ordination2d(meta_df, epoch="pre", dataset_name=dataset_name, ord_type="pca",
#             images_folder_path=plots_folder_path)



References
Auto-Encoding Variational Bayes https://arxiv.org/abs/1312.6114
Semi-Supervised Learning with Deep Generative Models https://arxiv.org/abs/1406.5298
Ladder Variational Autoencoders https://arxiv.org/abs/1602.02282
Auxiliary Deep Generative Models    https://arxiv.org/abs/1602.05473
Sylvester Normalizing Flows for Variational Inference  https://arxiv.org/abs/1803.05649
Improving Variational Auto-Encoders using Householder Flow https://arxiv.org/abs/1611.09630
Variational Inference with Normalizing Flows https://arxiv.org/abs/1505.05770
Convex combination linear IAF and the Householder Flow, J.M. Tomczak & M. Welling https://jmtomczak.github.io/deebmed.html

Github Ressources:
    https://github.com/wohlert/semi-supervised-pytorch
    https://github.com/jmtomczak/vae_vpflows
    https://github.com/jmtomczak/vae_householder_flow